# kXOR: Instance load Oracles

We define three oracles that load a kXOR instance, which are used in the algorithm.

We are given a kXOR instance $\mathcal{I}$ of $n$ variables,
with $\bar{m}$ unique scopes $\{U_j | j \in [\bar{m}]\}$.
We provide oracles to:
1. `LoadConstraintScopes`: Given $j \in [\bar{m}]$, compute $U_j$.
2. `LoadUniqueScopeIndex`: Given $U_j$, compute $j \in [\bar{m}]$
3. `PRGAUniqueConstraintRHS` Given $j$, apply $Rx(arccos(\sqrt{B_\mathcal{I}(S)/M}))$ on a target qubit.
(for an appropriate normalization $M$).

The first two oracles are independent of the RHS.
All these oracles can output arbitrary values for invalid inputs.

See :class:`KXorInstance` for the overall problem definition.

References:
    [Quartic quantum speedups for planted inference](https://arxiv.org/abs/2406.19378v1)
    Notation 2.24 for $B_\mathcal{I}$.
    Theorem 4.17, proof para 2 for $U_j$.

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `LoadConstraintScopes`
Given an index $j$, load the scope of the $j$-th unique constraint.

Given a $k$-XOR-SAT instance `inst` with $n$ variables and $m$ constraints.
Assuming `inst` has $\bar{m}$ unique constraints, we define $U_j \in {[n] \choose k}$
for $j \in [\bar{m}]$ as the $j$-th unique constraint scope.

See :class:`KXorInstance` for the overall problem definition.

The scopes are loaded using a QROM. If the input contains an invalid index, then
the output can be an arbitrary value.

#### Registers
 - `j`: a number in [\bar{m}]
 - `U`: $j$-th unique scope
 - `ancilla`: entangled intermediate qubits, to be uncomputed by the adjoint. 

#### References
 - [Quartic quantum speedups for planted inference](https://arxiv.org/abs/2406.19378v1). Theorem 4.17, proof para 2.


In [ ]:
from qualtran.bloqs.optimization.k_xor_sat import LoadConstraintScopes

### Example Instances

In [ ]:
import sympy

from qualtran.bloqs.optimization.k_xor_sat.kxor_instance import KXorInstance

n, m, k = sympy.symbols("n m k", positive=True, integer=True)
inst = KXorInstance.symbolic(n=n, m=m, k=k)
load_scopes_symb = LoadConstraintScopes(inst)

In [ ]:
from qualtran.bloqs.optimization.k_xor_sat.kxor_instance import Constraint, KXorInstance

inst = KXorInstance(
    n=6,
    k=4,
    constraints=(
        Constraint(S=(0, 1, 2, 3), b=1),
        Constraint(S=(0, 1, 4, 5), b=-1),
        Constraint(S=(1, 2, 4, 5), b=1),
        Constraint(S=(0, 3, 4, 5), b=1),
        Constraint(S=(2, 3, 4, 5), b=1),
        Constraint(S=(0, 1, 2, 3), b=1),
        Constraint(S=(0, 3, 4, 5), b=1),
        Constraint(S=(2, 3, 4, 5), b=1),
    ),
)
load_scopes = LoadConstraintScopes(inst)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([load_scopes_symb, load_scopes],
           ['`load_scopes_symb`', '`load_scopes`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
load_scopes_symb_g, load_scopes_symb_sigma = load_scopes_symb.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(load_scopes_symb_g)
show_counts_sigma(load_scopes_symb_sigma)